# Task 2: Analysing hospital 

**The task is to link the test dataset "Berlin Income" with hospital dataset of 2015 and calculate a bundle of descriptive statistical measures (Minimum, Maximum, Mean, Standard derivation) on bed numbers within a circle of 5 kilometers.**

## Load functions from SoRa R package
This steps are currently required to load all R functions from /R/ directory. In future, the SoRa R package will be installed directly.

In [ ]:
# load R functions from SoRa R Package
path <- "/home/jovyan/R/"
sora_functions  <- dir(path)
for (i in sora_functions) {
  source(paste0(path, i))
}

## Load, explore and prepare input survey data

In [ ]:
load("/home/jovyan/data/ber_income.rda")
head(ber_income)

In [ ]:
plot(ber_income$x, ber_income$y,
     xlab = "x", ylab = "y",
     main = "Households in Berlin",
     sub = "crs = 4647"
)
grid()

In [ ]:
# transform coordinate reference system (CRS) into LAEA (EPSG Code: 3035)
survey_data <- sora_custom(.data = ber_income, crs = 3035)

### Linking

In [ ]:
# define geospatial dataset of hospitals (including bed numbers) in 2015
spat_data <- sora_spatial(id = "<dataset_id>")

In [ ]:

linking <- sora_linking(
  method = "aggregate_attribute",
  selection_area = "circle",
  radius = 5000,
  output = c("mean", "min", "max", "sd")
)

In [ ]:
job_id <- sora_request(dataset = survey_data, link_to = spat_data, method = linking)

### Get linking result

In [ ]:
## get results
sora_calls <- sora_jobs()
job_id <- sora_calls$job_id[1]

if (sora_job_done(job_id)){
  results <- sora_results(job_id = job_id)
}
results <- sora_split_id(results, out_col = c("hid", "syear"))